In [1]:
import pandas as pd
import numpy as np

In [2]:
interaction = pd.read_csv ('interaction_table.csv', index_col=False)
interaction = interaction.iloc[:, [2,3]]

In [3]:
complex_comp

NameError: name 'complex_comp' is not defined

In [7]:
multi = pd.read_csv ('multidata_table.csv', index_col=None)
receptor = multi[multi['receptor'] == 1]
receptor = receptor.iloc[:, [0,1]]
ligand = multi[multi['receptor'] == 0]
ligand = ligand.iloc[:, [0,1]]

In [8]:
r=interaction[interaction['multidata_1_id'].isin(receptor['id_multidata'])].copy()
#fix column order for l-r
r[['multidata_1_id', 'multidata_2_id']] = r[['multidata_2_id', 'multidata_1_id']]
l=interaction[interaction['multidata_1_id'].isin(ligand['id_multidata'])].copy()

In [9]:
i=pd.concat([l, r], ignore_index=True).copy()

In [70]:
gene_table.columns=protein_name.columns

In [71]:
gene_table.columns

Index(['protein_name', 'protein_multidata_id'], dtype='object')

In [72]:
protein_name=gene_table

In [73]:
protein_name

,protein_name,protein_multidata_id
0,ABCA1,127
1,ACE2,1136
2,ACKR1,929
3,ACKR2,35
4,ACKR3,516
...,...,...
1615,XCL1,701
1616,XCL2,1225
1617,XCR1,689
1618,XPR1,1229


In [50]:
gene_table = pd.read_csv ('gene_table.csv', index_col=False)
gene_table = gene_table.iloc[:, [3,4]].copy()
protein_name = pd.read_csv ('protein_table.csv', index_col=False)
protein_name = protein_name.iloc[:, [1,5]].copy()
complex_comp = pd.read_csv ('complex_composition_table.csv', index_col=False)
complex_comp = complex_comp.iloc[:, [1,2]].copy()

In [74]:
i["ligand"] = ''
i["receptor"] = ''
i["l_adhesion_mol"] = ''
i["r_adhesion_mol"] = ''
i["l_complex"] = ''
i["r_complex"] = ''

In [75]:
df = pd.DataFrame(columns=['L', 'R'])
df["ligand"] = ''
df["receptor"] = ''
df["l_adhesion_mol"] = ''
df["r_adhesion_mol"] = ''

In [76]:
def split_complex(multi_id):
    ids=complex_comp.loc[complex_comp['complex_multidata_id'] == multi_id].protein_multidata_id.values
    return list(ids)

In [77]:
def get_name(prot_id):
    name=protein_name.loc[protein_name['protein_multidata_id'] == prot_id].protein_name
    name = name.str.replace('_HUMAN', '')
    return list(name)[0]

In [78]:
for x in range(len(i)):
    entry={'L':[],'R':[]}
    p_id = i.loc[x].multidata_1_id
    p_id2 = i.loc[x].multidata_2_id
    
    if p_id < 1282:
        if p_id in ligand.id_multidata.values:
            entry['L'].append(p_id)
        else:
            entry['R'].append(p_id)
    
    elif p_id > 1281:
        complex_ids = split_complex(p_id)
        
        for perid in complex_ids:
            if perid in ligand.id_multidata.values:
                entry['L'].append(perid)
            else:
                entry['R'].append(perid)
    
    if p_id2 < 1282:
        if p_id2 in ligand.id_multidata.values:
            entry['L'].append(p_id2)
        else:
            entry['R'].append(p_id2)
        
    elif p_id2 > 1281:
        complex_ids = split_complex(p_id2)
        
        for perid in complex_ids:
            if perid in ligand.id_multidata.values:
                entry['L'].append(perid)
            else:
                entry['R'].append(perid)
                
                
    if entry.get('L') and entry.get('R'):
        data=((x,y) for x in entry.get('L') for y in entry.get('R'))
        data=pd.DataFrame(data,columns=['L', 'R'])
        data['pid1']=p_id.astype('int')
        data['pid2']=p_id2.astype('int')
        data['l_adhesion_mol']=0
        data['r_adhesion_mol']=0
        df=pd.concat([df, data], ignore_index=True)
    else:
        if entry.get('L'):
            data=((x,y) for x in entry.get('L') for y in entry.get('L'))
            data=pd.DataFrame(data,columns=['L', 'R'])
            data['pid1']=p_id.astype('int')
            data['pid2']=p_id2.astype('int')
            data['l_adhesion_mol']=1
            data['r_adhesion_mol']=0
            df=pd.concat([df, data], ignore_index=True)
        else:
            data=((x,y) for x in entry.get('R') for y in entry.get('R'))
            data=pd.DataFrame(data,columns=['L', 'R'])
            data['pid1']=p_id.astype('int')
            data['pid2']=p_id2.astype('int')
            data['r_adhesion_mol']=1
            data['l_adhesion_mol']=0
            df=pd.concat([df, data], ignore_index=True)

In [79]:
for x in range(len(df)):
    l = df.loc[x].L
    r = df.loc[x].R
    l_name=get_name(l)
    r_name=get_name(r)
    df.at[x, 'ligand'] = l_name
    df.at[x, 'receptor'] = r_name

In [80]:
df

,L,R,ligand,receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2
0,901,917,PTGR1,LTB4R,0,0,1282.0,917.0
1,315,917,LTA4H,LTB4R,0,0,1430.0,917.0
2,901,1189,PTGR1,LTB4R2,0,0,1282.0,1189.0
3,315,1189,LTA4H,LTB4R2,0,0,1430.0,1189.0
4,315,864,LTA4H,GPR17,0,0,1430.0,864.0
...,...,...,...,...,...,...,...,...
4503,1177,418,CRLF2,IL7R,0,1,1500.0,1177.0
4504,1177,1177,CRLF2,CRLF2,0,1,1500.0,1177.0
4505,1177,1177,CRLF2,CRLF2,0,1,1500.0,1177.0
4506,1109,1090,NELL2,ROBO3,0,0,1109.0,1090.0


In [81]:
df.drop(df[df.ligand == df.receptor].index, inplace=True)

In [82]:
df = df.reset_index(drop=True)

In [83]:
df["pairs"]=df[["ligand", "receptor"]].apply("_".join, axis=1)

In [84]:
df[df.duplicated('pairs')]

,L,R,ligand,receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2,pairs
47,138,1252,WNT11,FZD10,0,0,1358.0,138.0,WNT11_FZD10
48,138,102,WNT11,LRP5,0,0,1359.0,138.0,WNT11_LRP5
50,138,108,WNT11,LRP6,0,0,1360.0,138.0,WNT11_LRP6
51,138,1260,WNT11,FZD1,0,0,1360.0,138.0,WNT11_FZD1
52,138,102,WNT11,LRP5,0,0,1361.0,138.0,WNT11_LRP5
...,...,...,...,...,...,...,...,...,...
3762,708,556,RXRG,CRABP2,1,0,1513.0,1486.0,RXRG_CRABP2
3763,556,698,CRABP2,ALDH1A3,1,0,1513.0,1486.0,CRABP2_ALDH1A3
3764,556,708,CRABP2,RXRG,1,0,1513.0,1486.0,CRABP2_RXRG
3766,418,1177,IL7R,CRLF2,0,1,1500.0,1177.0,IL7R_CRLF2


In [85]:
df=df.drop_duplicates('pairs')

In [86]:
df.to_csv('cpdb_revised.csv2', index=False)

In [87]:
df['receptor'].isnull().values.any()

False

In [88]:
cpdbnodes=list(df["ligand"])+list(df["receptor"])

In [89]:
"5HT1E" in cpdbnodes

False

In [90]:
"HTR1E" in cpdbnodes

True